In [39]:
import pandas as pd

In [40]:
df=pd.read_csv('FINAL.csv')


In [41]:
df.columns

Index(['match_id', 'innings', 'Unnamed: 2', 'season', 'start_date', 'venue',
       'ball', 'batting_team', 'bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes',
       'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'bowler_type',
       'Total_runs', 'Extraballs', 'over_type', 'number_of_wicket',
       'number_of_dotballs', 'Batting_type'],
      dtype='object')

In [42]:
df['innings'].unique()

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [43]:
df=df[(df['innings']==1) | (df['innings']==2)]

In [44]:
df['innings'].unique()

array([1, 2], dtype=int64)

In [45]:
win_data=df.groupby(['match_id','batting_team']).agg({'Total_runs':sum}).reset_index()
win_data

,match_id,batting_team,Total_runs
0,335982,Kolkata Knight Riders,222
1,335982,Royal Challengers Bangalore,82
2,335983,Chennai Super Kings,240
3,335983,Kings XI Punjab,207
4,335984,Delhi Daredevils,132
...,...,...,...
1683,1254084,Mumbai Indians,219
1684,1254085,Rajasthan Royals,220
1685,1254085,Sunrisers Hyderabad,165
1686,1254086,Delhi Capitals,167


In [46]:
#data=win_data.groupby('match_id').agg({'Total_runs':max})
#data
win_data=win_data.loc[win_data.groupby("match_id")["Total_runs"].idxmax()]

In [47]:
win_data.drop(['Total_runs'],axis=1,inplace=True)

In [48]:
final_data=pd.merge(df,win_data,on='match_id')

In [49]:
final_data['win_or_loss']=final_data.apply(lambda x: 'win'  if x['batting_team_x'] == x['batting_team_y'] else 'lose',axis=1)

In [50]:
final_data.columns

Index(['match_id', 'innings', 'Unnamed: 2', 'season', 'start_date', 'venue',
       'ball', 'batting_team_x', 'bowling_team', 'striker', 'non_striker',
       'bowler', 'runs_off_bat', 'extras', 'wides', 'noballs', 'byes',
       'legbyes', 'penalty', 'wicket_type', 'player_dismissed',
       'other_wicket_type', 'other_player_dismissed', 'bowler_type',
       'Total_runs', 'Extraballs', 'over_type', 'number_of_wicket',
       'number_of_dotballs', 'Batting_type', 'batting_team_y', 'win_or_loss'],
      dtype='object')

In [51]:
final_data['striker'].unique()

array(['SC Ganguly', 'BB McCullum', 'RT Ponting', 'DJ Hussey',
       'Mohammad Hafeez', 'R Dravid', 'W Jaffer', 'V Kohli', 'JH Kallis',
       'CL White', 'MV Boucher', 'B Akhil', 'AA Noffke', 'P Kumar',
       'Z Khan', 'SB Joshi', 'PA Patel', 'ML Hayden', 'MEK Hussey',
       'MS Dhoni', 'SK Raina', 'JDP Oram', 'S Badrinath', 'K Goel',
       'JR Hopes', 'KC Sangakkara', 'Yuvraj Singh', 'SM Katich',
       'IK Pathan', 'T Kohli', 'YK Pathan', 'SR Watson', 'M Kaif',
       'DS Lehmann', 'RA Jadeja', 'M Rawat', 'D Salunkhe', 'SK Warne',
       'SK Trivedi', 'G Gambhir', 'V Sehwag', 'S Dhawan', 'L Ronchi',
       'ST Jayasuriya', 'DJ Thornely', 'RV Uthappa', 'PR Shah',
       'AM Nayar', 'SM Pollock', 'Harbhajan Singh', 'S Chanderpaul',
       'LRPL Taylor', 'AC Gilchrist', 'Y Venugopal Rao', 'VVS Laxman',
       'A Symonds', 'RG Sharma', 'SB Styris', 'AS Yadav', 'SB Bangar',
       'WPUJC Vaas', 'RP Singh', 'WP Saha', 'LR Shukla',
       'DPMD Jayawardene', 'S Sohal', 'B Lee', 'PP Cha

In [52]:
batsman_name=input('enter the batsman name: ')

final_data['isOne'] = final_data['runs_off_bat'].apply(lambda x: 1 if x == 1 else 0)
final_data['isTwo'] = final_data['runs_off_bat'].apply(lambda x: 1 if x == 2 else 0)
final_data['isThree'] = final_data['runs_off_bat'].apply(lambda x: 1 if x == 3 else 0)
final_data['isFour'] = final_data['runs_off_bat'].apply(lambda x: 1 if x == 4 else 0)
final_data['isSix'] = final_data['runs_off_bat'].apply(lambda x: 1 if x == 6 else 0)
final_data['Boundary'] = final_data['runs_off_bat'].apply(lambda x: 1 if x == 6 or x==4 else 0)
        
batsman_data=final_data[final_data['striker']==batsman_name]
batsman_data=batsman_data.groupby(['win_or_loss']).agg({'runs_off_bat':'sum','match_id':'count','wicket_type':'count','number_of_dotballs':'sum','isFour':'sum','isSix':'sum','Boundary':'sum'}).reset_index().rename({'runs_off_bat':'Runs_scored','match_id':'Balls_faced','wicket_type':'dismissed','isFour':"Fours",'isSix':'Sixes'},axis=1)
batsman_data['Average']=batsman_data['Runs_scored']/batsman_data['dismissed']
batsman_data['Strike rate']=(batsman_data['Runs_scored']/batsman_data['Balls_faced'])*100
batsman_data['Dot ball percentage']=((batsman_data['number_of_dotballs']/batsman_data['Balls_faced'])*100).round()
batsman_data['Boundary percentage']=((batsman_data['Boundary']/batsman_data['Balls_faced'])*100).round()
batsman_data['Balls per boundary']=((batsman_data['Balls_faced']/batsman_data['Boundary']))
batsman_data=batsman_data.drop(['number_of_dotballs','Boundary'],axis=1)
batsman_data

enter the batsman name: RG Sharma


,win_or_loss,Runs_scored,Balls_faced,dismissed,Fours,Sixes,Average,Strike rate,Dot ball percentage,Boundary percentage,Balls per boundary
0,lose,1985,1643,90,180,69,22.055556,120.815581,37.0,15.0,6.598394
1,win,3495,2643,93,296,156,37.580645,132.236095,33.0,17.0,5.847345


In [53]:
final_data['wicket_type']=final_data['wicket_type'].fillna(0)


In [54]:
def is_bowlers_wicket(player_dismissed, wicket_type):
    if type(player_dismissed) == str:
        if wicket_type not in [ 'run out', 'retired hurt', 'obstructing the field',0]:
            return 1
        else:
            return 0
    else:
        return 0

In [55]:
final_data['wicket'] = final_data.apply(lambda x: is_bowlers_wicket(x['player_dismissed'], x['wicket_type']), axis = 1)

In [56]:
final_data['win_or_loss']=final_data.apply(lambda x: 'win'  if x['bowling_team'] == x['batting_team_y'] else 'lose',axis=1)


In [58]:
final_data['bowler'].unique()

array(['P Kumar', 'Z Khan', 'AA Noffke', 'JH Kallis', 'SB Joshi',
       'CL White', 'AB Dinda', 'I Sharma', 'AB Agarkar', 'SC Ganguly',
       'LR Shukla', 'B Lee', 'S Sreesanth', 'JR Hopes', 'IK Pathan',
       'K Goel', 'PP Chawla', 'WA Mota', 'JDP Oram', 'MS Gony',
       'M Muralitharan', 'P Amarnath', 'Joginder Sharma', 'GD McGrath',
       'B Geeves', 'MF Maharoof', 'R Bhatia', 'DL Vettori', 'MM Patel',
       'SR Watson', 'SK Trivedi', 'SK Warne', 'YK Pathan', 'D Salunkhe',
       'R Vinay Kumar', 'B Akhil', 'A Nehra', 'SM Pollock', 'DS Kulkarni',
       'ST Jayasuriya', 'Harbhajan Singh', 'AM Nayar', 'M Kartik',
       'Mohammad Hafeez', 'DJ Hussey', 'WPUJC Vaas', 'RP Singh',
       'SB Styris', 'SB Bangar', 'A Symonds', 'PP Ojha', 'Pankaj Singh',
       'Mohammad Asif', 'VY Mahesh', 'Shahid Afridi', 'DJ Bravo',
       'VS Yeligati', 'MA Khote', 'D Kalyankrishna', 'VRV Singh',
       'Sohail Tanvir', 'A Kumble', 'DNT Zoysa', 'SD Chitnis',
       'Yuvraj Singh', 'Shoaib Malik',

In [57]:
bowler_name=input('Enter the bowler name: ')
bowler_data=final_data[final_data['bowler']==bowler_name]
bowler_data=bowler_data.groupby(['win_or_loss']).agg({'runs_off_bat':'sum','match_id':'count','wicket':'sum','number_of_dotballs':'sum','number_of_dotballs':'sum','isFour':'sum','isSix':'sum','Boundary':'sum'}).reset_index().rename({'runs_off_bat':'Runs_scored','match_id':'Balls_bowled','wicket':'wickets','isFour':"Fours",'isSix':'Sixes'},axis=1)
bowler_data['bowling average']=bowler_data['Runs_scored']/bowler_data['wickets']
bowler_data['bowling strike-rate']=bowler_data['Balls_bowled']/bowler_data['wickets']
bowler_data['Dot ball percentage']=((bowler_data['number_of_dotballs']/bowler_data['Balls_bowled'])*100).round()
overs_bowled=[]
for i in bowler_data['Balls_bowled']:
    out1=i//6
    #print(str(out1))
    out2=i%6
    #print(out2)
    final=str(out1).split('.')[0]+'.'+str(out2).split('.')[0]
    overs_bowled.append(final)
bowler_data['overs_bowled']=overs_bowled
bowler_data['overs_bowled']=bowler_data['overs_bowled'].astype('float')
bowler_data['economy']=bowler_data['Runs_scored']/bowler_data['overs_bowled']

bowler_data=bowler_data.drop(['Balls_bowled','number_of_dotballs'],axis=1)
display(bowler_data)


Enter the bowler name: SL Malinga


,win_or_loss,Runs_scored,wickets,Fours,Sixes,Boundary,bowling average,bowling strike-rate,Dot ball percentage,overs_bowled,economy
0,lose,1311,48,149,28,177,27.312500,24.458333,39.0,195.4,6.709314
1,win,1883,122,165,58,223,15.434426,14.754098,38.0,300.0,6.276667
